Lanqing, Aug 21 2023

This notebook shows a minimal example of using matching method to do data causal inference. In this example, we did:

- Comparison of Kr83m S1 pulse shape SR0 VS SR1
- Comaprison of S1 pulse shape AmBe VS Ar37
- Comparison of S1 pulse shape AmBe VS Rn220
- Comparison of S1 pulse shape Ar37 VS Rn220

NB: to run this notebook, you probably need at least 16 GB RAM! 

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings('ignore')

import matching
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
import sys

# You need to git pull yourself: https://github.com/FaroutYLq/compeaks
sys.path.append('/home/yuanlq/xenon/compeaks')
import comparison

In [ ]:
import straxen
import strax
straxen.print_versions()

# Difference of S1 Pulse Shape SR0 VS SR1

In [ ]:
COMPARISON_SPACES1D = ['z', 's1_a_area', 's1_a_area_fraction_top', 's1_a_range_50p_area', 's1_a_range_90p_area',
                       's1_a_rise_time', 's1_a_n_channels', 's1_a_tight_coincidence', 's1_a_n_hits']
COMPARISON_SPACES2D = [('z', 's1_a_area_fraction_top'),
                       ('z', 's1_a_rise_time'), 
                       ('z', 's1_a_range_50p_area'),
                       ('z', 's1_a_range_90p_area'),
                       ('z', 's1_a_area'),
                       ('z', 's1_a_tight_coincidence'),
                       ('s1_a_area_fraction_top','s1_a_rise_time'),
                       ('s1_a_area', 's1_a_range_50p_area'),
                       ('s1_a_area', 's1_a_rise_time')]

## Data Loading

In [ ]:
kr_sr0 = np.load("/project2/lgrandi/yuanlq/shared/matching_examples/sr0_kr83m_DoubleS1SingleS2.npy")
kr_sr1aug = np.load("/project2/lgrandi/yuanlq/shared/matching_examples/sr1_kr83m_DoubleS1SingleS2_Aug2022.npy")

## Before Matching

In [ ]:
comparison.compare_1para(peak_extra0=kr_sr0,
                         peak_extra1=kr_sr1aug,
                         signal_type0='KrS1A SR0',
                         signal_type1='KrS1A SR1 (Aug 2022)',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18)

In [ ]:
comparison.compare_2para(peak_extra0=kr_sr0, 
                         peak_extra1=kr_sr1aug, 
                         signal_type0='KrS1A SR0', 
                         signal_type1='KrS1A SR1 (Aug 2022)', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

## After Matching

In [ ]:
inf = matching.inference.Inference(data=kr_sr0, simu=kr_sr1aug, 
                                   covariates=['s1_a_area_fraction_top', 's1_a_n_hits', 'z']);

In [ ]:
matched_kr_sr0 = inf.match_simu()
matched_kr_sr1aug = inf.simu

In [ ]:
comparison.compare_2para(peak_extra0=matched_kr_sr0, 
                         peak_extra1=matched_kr_sr1aug, 
                         signal_type0='Matched KrS1A SR0', 
                         signal_type1='Matched KrS1A SR1 (Aug 2022)', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=matched_kr_sr0,
                         peak_extra1=matched_kr_sr1aug,
                         signal_type0='KrS1A SR0',
                         signal_type1='KrS1A SR1 (Aug 2022)',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18)

# Can We See S1 Pulse Shape Difference in NR VS ER?

In [ ]:
COMPARISON_SPACES1D = ['z', 's1_area', 's1_area_fraction_top', 's1_range_50p_area', 's1_range_90p_area',
                       's1_rise_time', 's1_n_channels', 's1_tight_coincidence', 's1_n_hits']
COMPARISON_SPACES2D = [('z', 's1_area_fraction_top'),
                       ('z', 's1_rise_time'), 
                       ('z', 's1_range_50p_area'),
                       ('z', 's1_range_90p_area'),
                       ('z', 's1_area'),
                       ('z', 's1_tight_coincidence'),
                       ('s1_area_fraction_top','s1_rise_time'),
                       ('s1_area', 's1_range_50p_area'),
                       ('s1_area', 's1_rise_time')]

## Data Loading

In [ ]:
ambe_sr0 = np.load("/project2/lgrandi/yuanlq/shared/matching_examples/sr0_ambe_ss.npy")
ambe_sr0 = ambe_sr0[ambe_sr0['z']>-149]
ambe_sr0 = ambe_sr0[ambe_sr0['z']<-1]
ambe_sr0 = ambe_sr0[ambe_sr0['s1_area']<=50]
ambe_sr0 = pd.DataFrame(ambe_sr0)
""" Need to add SS cuts
daniel1 = np.load('/project2/lgrandi/wenz/strax_data/sr1/ambe/events_topCW5d9m_nv_coincidence_applied.npy')
daniel2 = np.load('/project2/lgrandi/wenz/strax_data/sr1/ambe/events_topCW7d8m_nv_coincidence_applied.npy')
ambe_sr1 = np.concatenate((daniel1, daniel2))
ambe_sr1 = ambe_sr1[ambe_sr1['z']>-150]
ambe_sr1 = ambe_sr1[ambe_sr1['z']<-0.1]
ambe_sr1 = ambe_sr1[ambe_sr1['s1_area']<50]
ambe_sr1 = pd.DataFrame(ambe_sr1)
"""

In [ ]:
# Get all SR0 Ar37
"""
import cutax
st = cutax.xenonnt_v8(output_folder="/project2/lgrandi/xenonnt/processed")
ar_runs = st.select_runs(run_mode='tpc_ar37', available="event_info").name
st.register_all(cutax.cut_lists.ar37)

available=[]
for run in ar_runs:
    if st.is_stored(run, 'event_info'):
        available.append(True)
    else:
        available.append(False)
available=np.array(available)
ar_runs = ar_runs[available]

for i,run in tqdm(enumerate(ar_runs)):
    if i==0:
        ar37 = st.get_array(run, ('event_info', 'cuts_ar37_kshell_s1s2'))
        ar37 = ar37[ar37['cuts_ar37_kshell_s1s2']]
    else:
        new = st.get_array(run, ('event_info', 'cuts_ar37_kshell_s1s2'))
        new = new[new['cuts_ar37_kshell_s1s2']]
        ar37 = np.concatenate((ar37, new))
    gc.collect()

np.save('/project2/lgrandi/yuanlq/shared/matching_examples/sr0_ar37_k.npy',ar37)
"""
ar37_sr0 = np.load('/project2/lgrandi/yuanlq/shared/matching_examples/sr0_ar37_k.npy')
ar37_sr0 = ar37_sr0[ar37_sr0['z']>-149]
ar37_sr0 = ar37_sr0[ar37_sr0['z']<-1]
ar37_sr0 = pd.DataFrame(ar37_sr0)

In [ ]:
"""
import cutax
st = cutax.xenonnt_offline()
rn_runs = st.select_runs(run_mode=('tpc_radon_hev', 'tpc_radon'), available="cuts_basic").name
st.register_all(cutax.cut_lists.rn220)

available=[]
for run in rn_runs:
    if st.is_stored(run, 'event_info'):
        available.append(True)
    else:
        available.append(False)
available=np.array(available)
rn_runs = rn_runs[available]

for i,run in tqdm(enumerate(rn_runs)):
    if i==0:
        rn220 = st.get_array(run, ('event_info', 'cuts_rn220'))
        rn220 = rn220[rn220['cuts_rn220']]
    else:
        new = st.get_array(run, ('event_info', 'cuts_rn220'))
        new = new[new['cuts_rn220']]
        rn220 = np.concatenate((rn220, new))
    gc.collect()
np.save('/project2/lgrandi/yuanlq/shared/matching_examples/sr1_rn220.npy',rn220)
"""

rn220_sr1 = np.load('/project2/lgrandi/yuanlq/shared/matching_examples/sr1_rn220.npy')
rn220_sr1 = rn220_sr1[rn220_sr1['z']>-149]
rn220_sr1 = rn220_sr1[rn220_sr1['z']<-1]
rn220_sr1 = pd.DataFrame(rn220_sr1)
rn220_sr1 = rn220_sr1[rn220_sr1['s1_area']<50]

In [ ]:
"""
import cutax
st = cutax.xenonnt_v8(_rucio_local_path='/project/lgrandi/rucio', include_rucio_local = True)
st.storage.append(strax.DataDirectory("/project2/lgrandi/xenonnt/processed/", readonly=True))
rn_runs = st.select_runs(run_mode=('tpc_radon_hev', 'tpc_radon'), available="event_info").name
st.register_all(cutax.cut_lists.rn220)

available=[]
for run in rn_runs:
    if st.is_stored(run, 'event_info'):
        available.append(True)
    else:
        available.append(False)
available=np.array(available)
rn_runs = rn_runs[available]

for i,run in tqdm(enumerate(rn_runs)):
    if i==0:
        rn220 = st.get_array(run, ('event_info', 'cuts_rn220'))
        rn220 = rn220[rn220['cuts_rn220']]
    else:
        new = st.get_array(run, ('event_info', 'cuts_rn220'))
        new = new[new['cuts_rn220']]
        rn220 = np.concatenate((rn220, new))
    gc.collect()
np.save('/project2/lgrandi/yuanlq/shared/matching_examples/sr0_rn220.npy',rn220)
"""

rn220_sr0 = np.load('/project2/lgrandi/yuanlq/shared/matching_examples/sr0_rn220.npy')
rn220_sr0 = rn220_sr0[rn220_sr0['z']>-149]
rn220_sr0 = rn220_sr0[rn220_sr0['z']<-1]
rn220_sr0 = pd.DataFrame(rn220_sr0)
rn220_sr0 = rn220_sr0[rn220_sr0['s1_area']<50]

In [ ]:
len(ar37_sr0)

In [ ]:
len(ambe_sr0)

In [ ]:
rn220 = pd.concat((rn220_sr0, rn220_sr1))
len(rn220)

## Before Matching

In [ ]:
comparison.compare_2para(peak_extra0=rn220, 
                         peak_extra1=ambe_sr0, 
                         signal_type0='Rn220 SR0+SR1', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_2para(peak_extra0=ar37_sr0, 
                         peak_extra1=ambe_sr0, 
                         signal_type0='Ar37 SR0', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=rn220,
                         peak_extra1=ambe_sr0,
                         signal_type0='Rn220 SR0+SR1',
                         signal_type1='SS AmBe SR0',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18
                         )

In [ ]:
comparison.compare_1para(peak_extra0=ar37_sr0,
                         peak_extra1=ambe_sr0,
                         signal_type0='Ar37 SR0',
                         signal_type1='SS AmBe SR0',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18
                         )

## After Matching

Here we treat Ar37 as "simulation" and SS AmBe as "data".

Matching: `s1_area_fraction_top`, `s1_n_hits`, `z`

### Match Rn220 to AmBe

In [ ]:
inf = matching.inference.Inference(data=ambe_sr0, simu=rn220, 
                                   covariates=['s1_area_fraction_top', 's1_n_hits', 'z']);
matched_ambe = inf.match_simu()
matched_rn220 = inf.simu

In [ ]:
comparison.compare_2para(peak_extra0=matched_rn220, 
                         peak_extra1=matched_ambe, 
                         signal_type0='Matched Rn220 SR0+SR1', 
                         signal_type1='Matched SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=matched_rn220,
                         peak_extra1=matched_ambe,
                         signal_type0='Matched Rn220 SR0+SR1', 
                         signal_type1='Matched SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18
                         )

### Match Rn220 to Ar37

In [ ]:
inf = matching.inference.Inference(data=ar37_sr0[:300000], simu=rn220, 
                                   covariates=['s1_area_fraction_top', 's1_n_hits', 'z']);
matched_ar37 = inf.match_simu()
matched_rn220 = inf.simu

In [ ]:
comparison.compare_2para(peak_extra0=matched_rn220, 
                         peak_extra1=matched_ar37, 
                         signal_type0='Matched Rn220 SR0+SR1', 
                         signal_type1='Matched Ar37 SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=matched_rn220, 
                         peak_extra1=matched_ar37, 
                         signal_type0='Matched Rn220 SR0+SR1', 
                         signal_type1='Matched Ar37 SR0', 
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18)

### Match AmBe to Ar37 

In [ ]:
inf = matching.inference.Inference(data=ambe_sr0, simu=ar37_sr0[:300000], 
                                   covariates=['s1_area_fraction_top', 's1_n_hits', 'z']);

In [ ]:
matched_ambe = inf.match_simu()
matched_ar37 = inf.simu

In [ ]:
comparison.compare_2para(peak_extra0=matched_ar37, 
                         peak_extra1=matched_ambe, 
                         signal_type0='Matched Ar37 SR0', 
                         signal_type1='Matched SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=matched_ar37,
                         peak_extra1=matched_ambe,
                         signal_type0='Ar37 SR0',
                         signal_type1='Matched SS AmBe SR0',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18
                         )

### Match Ar37 to AmBe

In [ ]:
inf = matching.inference.Inference(data=ar37_sr0[:300000], simu=ambe_sr0, 
                                   covariates=['s1_area_fraction_top', 's1_n_hits', 'z']);

In [ ]:
matched_ar37 = inf.match_simu()
matched_ambe = inf.simu

In [ ]:
comparison.compare_2para(peak_extra0= matched_ar37, 
                         peak_extra1= matched_ambe, 
                         signal_type0='Matched Ar37 SR0', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0= matched_ar37, 
                         peak_extra1= matched_ambe, 
                         signal_type0='Matched Ar37 SR0', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 18
                         )